# **Module 2: Vector Data in Python**
## **Prerequisites**
### Data
In this example, we will use the following datasets (exported from `spData` R package):  
- `world.shp` - World country polygons;
- `nz.shp` - The regions of New Zealand;
- `nz_height.shp` - High points in New Zealand;
- `seine.shp` - River network in France;
- `coffee_data.csv` - World coffee production statistics by country, 2016 and 2017.

### Software
To execute the code you will need a `conda` environment for Python with the packages imported below.

In [ ]:
import matplotlib.pyplot as plt
from shapely.geometry.polygon import Point, LineString, Polygon
import geopandas as gpd
import pandas as pd
import seaborn as sns
import random
import numpy as np
sns.set_style("whitegrid")

## **Vector Data Review**

Representation of the world using points, lines, and polygons

- Point: a precise location in space (e.g., weather stations)
- Line: a sequential collection of connected points (e.g., roads)
- Polygon: a region enclosed by lines (e.g., countries)

In [ ]:
print ("Below we create arbitrary shapes and plot them")
line1_x, lines1_y = LineString([(5, 5), (6, 6), (7, 10), (8, 11)]).xy
plt.plot(line1_x, lines1_y, color="steelblue")
line2_x, lines2_y = LineString([(8, 14), (9, 12)]).xy
plt.plot(line2_x, lines2_y, color="seagreen")

polygon = Polygon([(5, 7), (7, 3), (8, 4), (9, 8), (8, 9), (7, 15), (6, 14)])
polygon_x, polygon_y = polygon.exterior.xy
plt.fill(polygon_x, polygon_y, facecolor="khaki", edgecolor="black")

points = [Point(6, 9), Point(9, 14), Point(8, 8), Point(8.5, 9)]
points_x = [point.x for point in points]
points_y = [point.y for point in points]
plt.scatter(points_x, points_y, color="grey")

In [ ]:
print ("Different types of relationships between spatial features") 
polygon = Polygon([(-1, -1), (1, -1), (1, 1), (-1, -1)])
polygon_x, polygon_y = polygon.exterior.xy
plt.fill(polygon_x, polygon_y, facecolor="lightgray", edgecolor="red")

points = [Point(0.5, 0), Point(1, 1), Point(0, 1), Point(-1, 0.5)]
points_x = [point.x for point in points]
points_y = [point.y for point in points]
plt.scatter(points_x, points_y, edgecolor="black", facecolor="None", s=80)
annotations=["Point-1","Point-2","Point-3","Point-4"]
for i, label in enumerate(annotations):
    plt.annotate(label, (points_x[i]+0.03, points_y[i]+0.03))

In [ ]:
for i, label in enumerate(annotations):
    print ("Does polygon intersect with {}? - {}".format(label, polygon.intersects(points[i])))
for i, label in enumerate(annotations):
    print ("Does polygon contain {}? - {} ".format(label, polygon.contains(points[i])))
for i, label in enumerate(annotations):
    print ("Does polygon touch {}? - {} ".format(label, polygon.touches(points[i])))
for i, label in enumerate(annotations):
    print ("Is polygon within 0.9 distance to {}? - {} ".format(label, polygon.distance(points[i])<=0.9))

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(8,6))
for ax in [(0,0), (0,1), (1,0), (1,1)]:
    axs[ax].fill(polygon_x, polygon_y, facecolor="lightgray", edgecolor="red")
    axs[ax].scatter(points_x, points_y, color="grey")
    annotations=["Point-1","Point-2","Point-3","Point-4"]
    for i, label in enumerate(annotations):
        axs[ax].annotate(label, (points_x[i]+0.03, points_y[i]+0.03))

selected = [p for p in points if polygon.intersects(p)]
selected_x = [p.x for p in selected]
selected_y = [p.y for p in selected]
axs[0,0].scatter(selected_x, selected_y, color="blue")
axs[0,0].set_title("Points intersecting the polygon", weight="bold")

selected = [p for p in points if polygon.contains(p)]
selected_x = [p.x for p in selected]
selected_y = [p.y for p in selected]
axs[0,1].scatter(selected_x, selected_y, color="blue")
axs[0,1].set_title("Poins contained within the polygon", weight="bold")

selected = [p for p in points if polygon.touches(p)]
selected_x = [p.x for p in selected]
selected_y = [p.y for p in selected]
axs[1,0].scatter(selected_x, selected_y, color="blue")
axs[1,0].set_title("Poins touching the polygon", weight="bold")

selected = [p for p in points if polygon.distance(p)<=0.9]
selected_x = [p.x for p in selected]
selected_y = [p.y for p in selected]
axs[1,1].scatter(selected_x, selected_y, color="blue")
axs[1,1].set_title("Poins within 0.9 distance to the polygon", weight="bold")
plt.tight_layout()

## **Attribute operations**

In [ ]:
world = gpd.read_file("./data-module-2/world.shp")
world.head()

### Select by attributes

In [ ]:
midsized_countries = world[(world.area_km2 > 1000000)&(world.area_km2 < 5000000)]
midsized_countries.head()

In [ ]:
fig, axs = plt.subplots(2,1, figsize=(14,14))
world.plot(ax=axs[0], column="iso_a2", cmap="Pastel2", edgecolor="grey")
axs[0].set_title("All countries", weight="bold")
world.plot(ax=axs[1], color="whitesmoke", edgecolor="grey")
midsized_countries.plot(ax=axs[1], facecolor="palevioletred", edgecolor="grey")
axs[1].set_title("Mid-sized countries", weight="bold")

### Dissolve  by attributes

In [ ]:
continents = world[["continent", "geometry", "pop"]].dissolve(by="continent", aggfunc="sum")
continents

In [ ]:
fig, axs = plt.subplots(2,1, figsize=(14,14))
world["pop"] = world["pop"].fillna(0)
world.plot(ax=axs[0], column="pop", legend=True, scheme="User_Defined", cmap="YlOrBr", 
           edgecolor="grey", classification_kwds=dict(bins=[33000000,100000000,204000000,319000000]),
           legend_kwds={"labels": ["< 33 mln", "33 - 100 mln", "100 - 204 mln", 
                                   "204 - 319 mln", "> 319 mln"], "bbox_to_anchor":(0.17, 0.4)})
axs[0].set_title("Population by countries", weight="bold")
axs[0].set_facecolor("powderblue")
continents.plot(ax=axs[1], column="pop", cmap="afmhot_r", edgecolor="grey")
continents.plot(ax=axs[1], column="pop", legend=True, scheme="User_Defined", cmap="YlOrBr", 
           edgecolor="grey", classification_kwds=dict(bins=[38000000,413000000,670000000,1155000000]),
           legend_kwds={"labels": ["< 38 mln", "38 - 413 mln", "413 - 670 mln", "670 - 1,155 mln",
                                   "> 1,155 mln"], "bbox_to_anchor":(0.18, 0.4)})
axs[1].set_title("Population by continents", weight="bold")
axs[1].set_facecolor("powderblue")

### Join by attributes

In [ ]:
coffee_data = pd.read_csv("./data-module-2/coffee_data.csv")
coffee_data.head()

In [ ]:
world_coffee = world.merge(coffee_data, on="name_long")
world_coffee.head()

In [ ]:
fig, ax = plt.subplots(figsize=(14,7))
world.plot(ax=ax, color="whitesmoke", edgecolor="grey")
world_coffee.plot(ax=ax, column="coffee_production_2017", cmap="copper_r", edgecolor="grey",
                 scheme="FisherJenks", legend=True)
ax.set_facecolor("powderblue")
ax.set_title("2017 coffee production", weight="bold")

## **Spatial operations**
### Spatial joins (within)

In [ ]:
nz = gpd.read_file("./data-module-2/nz.shp")
nz_height = gpd.read_file("./data-module-2/nz_height.shp")
canterbury = nz[nz.Name == "Canterbury"]
canterbury_height = gpd.sjoin(nz_height, canterbury, predicate="within")

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(9,9))
nz.plot(ax=axs[0], edgecolor="dimgrey", facecolor="whitesmoke")
nz_height.plot(ax=axs[0], marker = "^", facecolor="None", edgecolor="red", markersize=120)
axs[0].set_title("High Points in New Zealand", weight="bold")
axs[0].set_facecolor("powderblue")
nz.plot(ax=axs[1], edgecolor="dimgrey", facecolor="whitesmoke")
canterbury.plot(ax=axs[1], edgecolor="dimgrey", facecolor="lightgrey")
canterbury_height.plot(ax=axs[1], marker = "^", facecolor="None", edgecolor="red", markersize=120)
axs[1].set_title("High Points in Canterbury", weight="bold")
axs[1].set_facecolor("powderblue")
plt.tight_layout()

### Spatial joins (within, contains)

In [ ]:
minx, miny, maxx, maxy = world.total_bounds
random.seed(200)
points = []
while len(points) < 10:
    pnt = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
    if world.contains(pnt).any():
        points.append(pnt)
points_gpd = gpd.GeoDataFrame(geometry=points, crs="epsg:4326")
points_gpd.head()

In [ ]:
points_within_world = gpd.sjoin(points_gpd, world, predicate="within")
points_within_world.head()

In [ ]:
world_with_points = gpd.sjoin(world, points_gpd, predicate="contains")
world_with_points.head()

In [ ]:
fig, axs = plt.subplots(2,1, figsize=(14,14))
world.plot(ax=axs[0], edgecolor="slategrey", facecolor="none")
points_within_world.plot(ax=axs[0], marker="*", markersize=130, column="name_long", legend=True, cmap="Paired")
axs[0].set_title("Random points within countries", weight="bold")
world.plot(ax=axs[1], edgecolor="slategrey", facecolor="none")
world_with_points.plot(ax=axs[1], column="name_long", legend=True, cmap="Paired")
axs[1].set_title("Countries containing random points", weight="bold")

### Combining operations (spatial join + dissolve)

In [ ]:
nz_merged = gpd.sjoin(nz, nz_height, predicate="contains")
nz_merged = nz_merged[["Name", "geometry", "elevation"]].dissolve(by=["Name"], aggfunc="mean")
nz_merged.head()

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(9,9))
nz.plot(ax=axs[0], edgecolor="dimgrey", facecolor="whitesmoke")
nz_height.plot(ax=axs[0], marker = "o", markersize=100, alpha=0.6, column="elevation", legend=True, scheme="FisherJenks", cmap="magma_r")
axs[0].set_title("Elevation of High Points in New Zealand", weight="bold")
axs[0].set_facecolor("powderblue")
nz.plot(ax=axs[1], edgecolor="dimgrey", facecolor="whitesmoke")
nz_merged.plot(ax=axs[1], column="elevation", edgecolor="dimgrey", legend=True,  scheme="FisherJenks", cmap="magma_r")
axs[1].set_title("Average elevation of High Points in New Zealand", weight="bold")
axs[1].set_facecolor("powderblue")
plt.tight_layout()

### Centroids

In [ ]:
nz_centroid = nz.copy()
nz_centroid["geometry"]=nz.centroid
nz_centroid.head()

In [ ]:
seine = gpd.read_file("./data-module-2/seine.shp")
seine_centroid = seine.copy()
seine_centroid["geometry"]=seine.centroid
seine_centroid

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(18,9))
nz.plot(ax=axs[0], column="Name", cmap="Pastel1")
nz_centroid.plot(ax=axs[0], edgecolor="dimgrey", column="Name", cmap="Pastel1")
axs[0].set_title("New Zealand Regions with Centroids", weight="bold",  fontsize=15)
axs[0].set_facecolor("powderblue")

seine.plot(ax=axs[1], column="name", cmap="Accent")
seine_centroid.plot(ax=axs[1], edgecolor="dimgrey", column="name", cmap="Accent", markersize=120)
axs[1].set_title("Seine Tributaries with Centroids", weight="bold",  fontsize=15)
axs[1].set_facecolor("ivory")
plt.tight_layout()

### Buffer

In [ ]:
nz_buffer = nz.copy()
nz_buffer["geometry"]=nz.buffer(50000)
seine_buffer = seine.copy()
seine_buffer["geometry"]=seine.buffer(5000)
seine_buffer

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(18,9))
nz_buffer.plot(ax=axs[0], edgecolor="white", column="Name", cmap="Pastel1", alpha=0.7)
nz.plot(ax=axs[0], facecolor="none", edgecolor="dimgrey")
axs[0].set_title("New Zealand Regions with 50km buffer", weight="bold", fontsize=15)
axs[0].set_facecolor("powderblue")

seine_buffer.plot(ax=axs[1], edgecolor="dimgrey", column="name", cmap="Accent")
seine.plot(ax=axs[1], color="white")
axs[1].set_title("Seine Tributaries with 5km Buffer", weight="bold", fontsize=15)
axs[1].set_facecolor("ivory")
plt.tight_layout()

## **Exercices**
### Data
- `gw_provinces_extra.shp` - Minnesota Groundwater Provinces shapefile;
- `mn_rivers.shp` - Minnesota river shapefile;
- `soil_samp.shp` - Soil temperature stations shapefile.

**Question 1. Find and plot all groundwater provinces in Minnesota that have `var1` equal to `"a"` and `var3` greater than 100.**

**Question 2. Create a simple `DataFrame` with the following code:**
```r
random.seed(0) 
data = {"ProvID": list(range(1,7)),
        "var4":["yes", "no", "no", "yes", "np", "yes"],
        "var5": random.sample(range(10, 50), 6)}
gw_prov_df = pd.DataFrame(data)
```
**Merge `gw_provinces_extra_gdf` with `gw_prov_df` into a new object called `gw_prov_stats`. Which columns were used as keys for join? Pay attention to the data type of these columns.**

**Question 3. Dissolve `var3` as `sum` by `var1`.**

**Question 4. Find and plot the rivers found (even partially) in groundwater PROVINCE 5.**

**Question 5. Which groundwater province does the Lamberton (soil sample) site located in?**

**Question 6. Find the soil sample sites that are within 10 km of the Mississippi river.**

**Question 7. Find 10-km buffer for Mississippi river, the centroid for Mississippi river, and Mississippi river segments that fall within PROVINCE 4. Display results on a figure with 3 subplots.**